In [ ]:
# | default_exp _components.helpers

# Internal helpers

In [ ]:
# | export


def in_notebook():
    try:
        from IPython import get_ipython

        if "IPKernelApp" not in get_ipython().config:
            return False
    except ImportError:
        return False
    except AttributeError:
        return False
    return True

In [ ]:
in_notebook()

True

In [ ]:
# | export

import textwrap
from functools import wraps
from inspect import signature
from typing import *

import docstring_parser
from fastcore.meta import delegates

if in_notebook():
    from tqdm.notebook import tqdm, trange
else:
    from tqdm import tqdm, trange

from fastkafka._components.logger import get_logger, supress_timestamps

In [ ]:
from aiokafka import AIOKafkaConsumer, AIOKafkaProducer
from nbdev_mkdocs.docstring import run_examples_from_docstring

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
supress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
# | export

F = TypeVar("F", bound=Callable[..., Any])


def _format_args(xs: List[docstring_parser.DocstringParam]) -> str:
    return "\nArgs:\n - " + "\n - ".join(
        [f"{x.arg_name} ({x.type_name}): {x.description}" for x in xs]
    )


def combine_params(f: F, o: Union[Type, Callable[..., Any]]) -> F:
    """Combines docstring arguments of a function and another object or function

    Args:
        f: destination functions where combined arguments will end up
        o: source function from which arguments are taken from

    Returns:
        Function f with augumented docstring including arguments from both functions/objects
    """
    src_params = docstring_parser.parse_from_object(o).params
    #     logger.info(f"combine_params(): source:{_format_args(src_params)}")
    docs = docstring_parser.parse_from_object(f)
    #     logger.info(f"combine_params(): destination:{_format_args(docs.params)}")
    dst_params_names = [p.arg_name for p in docs.params]

    combined_params = docs.params + [
        x for x in src_params if not x.arg_name in dst_params_names
    ]
    #     logger.info(f"combine_params(): combined:{_format_args(combined_params)}")

    docs.meta = [
        x for x in docs.meta if not isinstance(x, docstring_parser.DocstringParam)
    ] + combined_params  # type: ignore

    f.__doc__ = docstring_parser.compose(
        docs, style=docstring_parser.DocstringStyle.GOOGLE
    )
    return f

In [ ]:
def f2(a: int, b: str):
    """
    Args:
        a: parameter a
        b: parameter bbbb
    """


def f1(b: str, c: int):
    """Function f1
    Args:
        b: parameter b
        c: parameter c

    Raises:
        ValueError: sometimes
    """


combine_params(f1, f2).__doc__

expected = """Function f1
Args:
    b: parameter b
    c: parameter c
    a: parameter a

Raises:
    ValueError: sometimes"""

assert f1.__doc__ == expected

In [ ]:
# | export


def delegates_using_docstring(o: Union[Type, Callable[..., Any]]) -> Callable[[F], F]:
    def _delegates_using_docstring(f: F) -> F:
        def _combine_params(o: Union[Type, Callable[..., Any]]) -> Callable[[F], F]:
            def __combine_params(f: F, o: Union[Type, Callable[..., Any]] = o) -> F:
                return combine_params(f=f, o=o)

            return __combine_params

        @_combine_params(o)
        @delegates(o)
        @wraps(f)
        def _f(*args, **kwargs):
            return f(*args, **kwargs)

        return _f

    return _delegates_using_docstring

In [ ]:
def f2(a: str, d: int) -> None:
    """
    Args:
        a: parameter a
        b: parameter bbbb
    """
    pass


@delegates_using_docstring(f2)
def f1(b: str, c: int, **kwargs):
    """Function f1
    Args:
        b: parameter b
        c: parameter c

    Raises:
        ValueError: sometimes
    """
    pass


expected = """Function f1
Args:
    b: parameter b
    c: parameter c
    a: parameter a

Raises:
    ValueError: sometimes"""
assert f1.__doc__ == expected

In [ ]:
signature(f1).parameters, signature(f2).parameters

In [ ]:
@delegates(f2)
def f3(b: str, c: int, **kwargs):
    """Function f1
    Args:
        b: parameter b
        c: parameter c

    Raises:
        ValueError: sometimes
    """
    pass


signature(f3).parameters

In [ ]:
@delegates_using_docstring(AIOKafkaConsumer)
def f(a: int, **kwargs) -> str:
    """function a
    Args:
        a: parameter a

    Returns:
        stuff
    """
    print(f"{a=}")

In [ ]:
print(f.__doc__)

In [ ]:
signature(f).parameters

In [ ]:
# | export


def use_parameters_of(
    o: Union[Type, Callable[..., Any]], **kwargs: Dict[str, Any]
) -> Dict[str, Any]:
    """Restrict parameters passwed as keyword arguments to parameters from the signature of ``o``

    Args:
        o: object or callable which signature is used for restricting keyword arguments
        kwargs: keyword arguments

    Returns:
        restricted keyword arguments

    """
    allowed_keys = set(signature(o).parameters.keys())
    return {k: v for k, v in kwargs.items() if k in allowed_keys}

In [ ]:
assert use_parameters_of(AIOKafkaConsumer, api_version=0.1, radnom_param="random") == {
    "api_version": 0.1
}